In [8]:
import csv
import base64
import os
import requests
from ipywidgets import widgets, interact, VBox, Layout
from IPython.display import display

# Configuration
API_URL = "https://api.freepik.com/v1/ai/text-to-image"
INPUT_CSV = "image_prompt_chunks.csv"
OUTPUT_DIR = "Generated Image"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Widgets for parameter selection
guidance_scale = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Guidance Scale:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width
)
image_size = widgets.Dropdown(
    options=[
        "square_1_1", "classic_4_3", "traditional_3_4", "widescreen_16_9",
        "social_story_9_16", "smartphone_horizontal_20_9", "smartphone_vertical_9_20",
        "standard_3_2", "portrait_2_3", "horizontal_2_1", "vertical_1_2",
        "social_5_4", "social_post_4_5"
    ],
    value="square_1_1",
    description="Image Size:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)
num_images = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    step=1,
    description="Number of Images:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)
seed = widgets.IntSlider(
    value=-1,
    min=-1,
    max=1000000,
    step=1,
    description="Seed (-1 for random):",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)
color = widgets.Dropdown(
    options=[
        None, "b&w", "pastel", "sepia", "dramatic", "vibrant", "orange&teal", "film-filter",
        "split", "electric", "pastel-pink", "gold-glow", "autumn", "muted-green",
        "deep-teal", "duotone", "terracotta&teal", "red&blue", "cold-neon", "burgundy&blue"
    ],
    value=None,
    description="Color:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)
framing = widgets.Dropdown(
    options=[
        None, "portrait", "macro", "panoramic", "aerial-view", "close-up", "cinematic",
        "high-angle", "low-angle", "symmetry", "fish-eye", "first-person"
    ],
    value=None,
    description="Framing:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)
lighting = widgets.Dropdown(
    options=[
        None, "studio", "warm", "cinematic", "volumetric", "golden-hour", "long-exposure",
        "cold", "iridescent", "dramatic", "hardlight", "redscale", "indoor-light"
    ],
    value=None,
    description="Lighting:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)
style = widgets.Dropdown(
    options=[
        None, "photo", "digital-art", "3d", "painting", "low-poly", "pixel-art", "anime",
        "cyberpunk", "comic", "vintage", "cartoon", "vector", "studio-shot", "dark",
        "sketch", "mockup", "2000s-pone", "70s-vibe", "watercolor", "art-nouveau",
        "origami", "surreal", "fantasy", "traditional-japan"
    ],
    value=None,
    description="Style:",
    layout=Layout(width='500px'),  # Adjust label width
    style={'description_width': '200px'}  # Adjust description width  # Adjust label width
)

# Button to trigger image generation
run_button = widgets.Button(description="Generate Images", button_style="success")
output = widgets.Output()

# Function to process the CSV and generate images
def process_csv(input_csv, output_dir, params):
    try:
        with open(input_csv, mode="r", encoding="windows-1252") as csv_file:
            reader = csv.DictReader(csv_file)

            if "image_prompt" not in reader.fieldnames:
                with output:
                    print(f"The file {input_csv} does not contain an 'image_prompt' column.")
                return

            for idx, row in enumerate(reader, start=1):
                prompt = row["image_prompt"]
                output_path = os.path.join(output_dir, f"chunk_{idx}.png")
                generate_image(prompt, output_path, params)
    except FileNotFoundError:
        with output:
            print(f"File {input_csv} not found.")
    except Exception as e:
        with output:
            print(f"Error processing the file {input_csv}: {e}")

# Function to generate an image using the API
def generate_image(prompt, output_path, params):
    payload = {
        "prompt": prompt,
        "guidance_scale": params["guidance_scale"],
        "image": {"size": params["image_size"]},
        "num_images": params["num_images"],
        "seed": params["seed"],
        "styling": params["styling"],
    }

    # Remove None values from styling
    payload["styling"] = {k: v for k, v in payload["styling"].items() if v}

    try:
        response = requests.post(
            API_URL,
            headers={
                "x-freepik-api-key": "YOUR API KEY",  # Replace with your API key
                "Content-Type": "application/json"
            },
            json=payload
        )
        response.raise_for_status()
        data = response.json()

        # Process image data
        if "data" in data and data["data"]:
            image_base64 = data["data"][0]["base64"]
            image_bytes = base64.b64decode(image_base64)

            # Save the image
            with open(output_path, "wb") as img_file:
                img_file.write(image_bytes)
            with output:
                print(f"Image saved: {output_path}")
        else:
            with output:
                print(f"Failed to generate image for prompt: {prompt}")

    except requests.RequestException as e:
        with output:
            print(f"API request failed for prompt {prompt}: {e}")
    except Exception as e:
        with output:
            print(f"Error generating image for prompt {prompt}: {e}")

# Event handler for the run button
def on_button_click(b):
    params = {
        "guidance_scale": guidance_scale.value,
        "image_size": image_size.value,
        "num_images": num_images.value,
        "seed": seed.value,
        "styling": {
            "color": color.value,
            "framing": framing.value,
            "lightning": lighting.value,
            "style": style.value
        }
    }
    output.clear_output()
    process_csv(INPUT_CSV, OUTPUT_DIR, params)

# Attach the event handler to the button
run_button.on_click(on_button_click)

# Display the widgets and the button
display(VBox([guidance_scale, image_size, num_images, seed, color, framing, lighting, style, run_button, output]))